# Post-Mix Analysis Pipeline - Cleaned Up Version

This notebook provides a streamlined end-to-end audio post-production pipeline:
1. Audio import and analysis
2. Pre-mastering preparation 
3. Mastering with different styles
4. Streaming platform simulation
5. Comparison reporting

**Key improvements in this version:**
- Centralized configuration system
- Proper error handling
- Eliminated code duplication
- Consistent naming conventions
- Modular, reusable components

In [1]:
# Import all required modules with cleaned up dependencies
from __future__ import annotations

# Core modules 
import os
import json
import numpy as np
import soundfile as sf
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import asdict

# Project modules (now cleaned up)
from config import CONFIG, CFG  # New centralized configuration
from audio_utils import validate_audio  # New utilities
from utils import ensure_audio_valid  # Audio validation from utils
from data_handler import *
from analysis import *
from dsp_premitives import *
from processors import *
from render_engine import *
from pre_master_prep import *
from streaming_normalization_simulator import *
from comparison_reporting import *
from presets_recommendations import *
from logging_versioning import *
from utils import *
from mastering_orchestrator import *

print("✓ All modules loaded successfully with improved error handling and configuration")
print(f"✓ Using configuration version: {CONFIG.to_dict()['audio']}")
print(f"✓ Workspace configured: {CONFIG.workspace.get_workspace_root()}")

Post-Mix I/O layer loaded: AudioBuffer, load_wav, save_wav, resample_poly, slice_preview, with_suffix, auto_out_path, sha256_file, print_audio_summary.
Convenience helpers loaded: make_workspace, batch_load_wavs, env_fingerprint, Manifest, write/read_manifest, register_input, register_artifact, import_mix.
Analysis layer loaded: analyze_wav/analyze_audio_array, analysis_table, plot_spectrum, plot_short_term_loudness, plot_waveform_excerpt, LUFS approx, true-peak approx, stereo & health metrics.
DSP Primitives Layer loaded:
- Gain/level: apply_gain_db, normalize_peak, normalize_lufs, measure_peak, measure_rms
- Filters: highpass_filter, lowpass_filter, bandpass_filter, shelf_filter, peaking_eq, notch_filter, tilt_eq
- Stereo: mid_side_encode, mid_side_decode, stereo_widener
- Dynamics: compressor (soft‑knee), transient_shaper
- Fades: fade_in, fade_out
- K‑weighting/LUFS approx: k_weight, lufs_integrated_approx
Patched: render_from_cache now unpacks widen_stereo tuple correctly.
Process

## Configuration

Set your input file path and processing options here. The system now uses centralized configuration for consistency.

In [2]:
# ============================================
# CONFIGURATION - Edit these values as needed  
# ============================================

# Input file (CHANGE THIS to your file path)
MIX_SOURCE_PATH = "/Users/itay/Documents/post_mix_data/mixes/generativist/ITAY - Generativist v.STEFAN v4.wav"

# Project settings
PROJECT_NAME = "postmix_cleaned_v1"

# Processing options
ENABLE_STREAMING_SIMULATION = True
MAX_RECOMMENDATIONS = 3
LEVEL_MATCH_LUFS = -14.0  # For A/B comparison previews
REFERENCE_NAME = "Original"

# Validate configuration
print(f"Project: {PROJECT_NAME}")
print(f"Input file: {MIX_SOURCE_PATH}")
print(f"📁 Workspace root: {CONFIG.workspace.get_workspace_root()}")
print(f"Audio config: {CONFIG.audio.default_bit_depth} @ {CONFIG.audio.render_peak_target_dbfs} dBFS peak target")
print(f"Processing config: Limiter attack={CONFIG.processing.limiter_attack_ms}ms, release={CONFIG.processing.limiter_release_ms}ms")
print(f"\n📍 IMPORTANT: All outputs will be saved to {CONFIG.workspace.get_workspace_root()}")
print("   This keeps your Git repo clean and prevents accidentally committing large audio files!")

Project: postmix_cleaned_v1
Input file: /Users/itay/Documents/post_mix_data/mixes/generativist/ITAY - Generativist v.STEFAN v4.wav
📁 Workspace root: /Users/itay/Documents/post_mix_data/PostMixRuns
Audio config: PCM_24 @ -1.0 dBFS peak target
Processing config: Limiter attack=1.0ms, release=50.0ms

📍 IMPORTANT: All outputs will be saved to /Users/itay/Documents/post_mix_data/PostMixRuns
   This keeps your Git repo clean and prevents accidentally committing large audio files!


## Pipeline Execution

Run the complete audio processing pipeline with improved error handling and logging.

In [3]:
# ============================================
# MAIN PROCESSING PIPELINE
# ============================================

try:
    # 1) Setup workspace and logging
    print("📁 Setting up workspace...")
    workspace_paths = make_workspace(project=PROJECT_NAME)
    manifest = Manifest(project=PROJECT_NAME, workspace=workspace_paths)
    
    logger = RunLogger.start(workspace_paths.root, tag=PROJECT_NAME)
    environment_info = capture_environment()
    logger.log_event("env", {"environment": environment_info})
    
    # 2) Import and validate input audio
    print("🎵 Importing audio...")
    if not os.path.exists(MIX_SOURCE_PATH):
        raise FileNotFoundError(f"Audio file not found: {MIX_SOURCE_PATH}")
        
    mix_path = import_mix(workspace_paths, MIX_SOURCE_PATH, alias="mix.wav")
    register_input(manifest, mix_path, alias="mix")
    register_and_log_artifact(manifest, logger, mix_path, kind="input", 
                             params={"alias": "mix"}, stage="import_mix")
    
    # Load and validate audio
    audio_data, sample_rate = sf.read(mix_path)
    ensure_audio_valid(audio_data, "input mix")
    
    # Print summary
    audio_buffer = load_wav(mix_path)
    print_audio_summary(audio_buffer, "Original Mix")
    
    print("✅ Audio imported and validated successfully")
    
except Exception as e:
    print(f"❌ Setup failed: {e}")
    raise

📁 Setting up workspace...
Workspace created at: /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_cleaned_v1_20250826-093714
🎵 Importing audio...
Imported mix → /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_cleaned_v1_20250826-093714/inputs/mix.wav
Original Mix: sr=44100 | ch=2 | dur=248.399s | peak=0.910527 | rms=0.125212
  path: /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_cleaned_v1_20250826-093714/inputs/mix.wav
  sha256: 66a33b29fb112fc0...
  src dtype: int32 | src ch: 2
✅ Audio imported and validated successfully


In [4]:
# ============================================
# ANALYSIS AND RECOMMENDATIONS
# ============================================

try:
    # 3) Analyze original mix
    print("📊 Analyzing audio...")
    original_analysis = analyze_wav(mix_path)
    
    # Log detailed metrics
    logger.log_metrics("analysis_original", {
        "sr": original_analysis.sr, 
        "duration_s": original_analysis.duration_s,
        "peak_dbfs": original_analysis.basic["peak_dbfs"],
        "true_peak_dbfs": original_analysis.true_peak_dbfs,
        "rms_dbfs": original_analysis.basic["rms_dbfs"],
        "lufs_integrated": original_analysis.lufs_integrated,
        "crest_db": original_analysis.basic["crest_db"],
        "bass_energy_%": original_analysis.bass_energy_pct,
        "air_energy_%": original_analysis.air_energy_pct,
        "phase_correlation": original_analysis.stereo["phase_correlation"],
        "stereo_width": original_analysis.stereo["stereo_width"],
        "spectral_flatness": original_analysis.spectral_flatness,
    })
    
    # 4) Generate processing recommendations
    print("🎯 Generating recommendations...")
    recommendations = recommend_from_analysis(original_analysis)
    print(recommendation_summary(recommendations))
    
    # Build comprehensive processing plan with ALL variants
    processing_plan = build_all_variants_plan(
        recommendations, prefix="PM", top_recommendations=MAX_RECOMMENDATIONS
    )
    
    logger.log_params("recommendations", {
        "plan": [(name, asdict(dial_state)) for (name, dial_state) in processing_plan]
    }, code_versions={"presets_recs": CODE_VERSIONS["presets_recs"]})
    
    print(f"✅ Generated {len(processing_plan)} processing variants (ALL presets + recommendations)")
    print(f"📋 Variants include: Transparent, Bassy, Punchy, Clear, Airy, Wide, Genre-specific, and more!")
    print(f"⭐ Top {MAX_RECOMMENDATIONS} recommendations marked with '_recommended' suffix")
    
except Exception as e:
    print(f"❌ Analysis failed: {e}")
    raise

📊 Analyzing audio...
🎯 Generating recommendations...
- Punch Up the Transients (priority 2): B16 P38 C10 A14 W8
    • crest 17.4 dB (dynamic) and LUFS -19.2 (quiet) → add transient definition
    • small air lift to help intelligibility
- Widen Image (Safe) (priority 4): B14 P14 C10 A12 W18
    • stereo_width 0.31 is narrow → add width
    • tone moves kept gentle to avoid destabilizing center
    ↪ If mono compatibility is critical, keep width ≤ 15.
✅ Generated 37 processing variants (ALL presets + recommendations)
📋 Variants include: Transparent, Bassy, Punchy, Clear, Airy, Wide, Genre-specific, and more!
⭐ Top 3 recommendations marked with '_recommended' suffix


In [5]:
# ============================================
# PRE-MASTERING PROCESSING
# ============================================

try:
    # 5) Pre-mastering preparation - VARIANTS ONLY (no single premaster folder)
    print("🔧 Pre-mastering preparation - generating ALL variants...")
    
    # Setup render engine with improved configuration
    engine = RenderEngine(audio_data, sample_rate, 
                         preprocess=PreprocessConfig(
                             low_cutoff=CONFIG.audio.prep_hpf_hz,
                             kick_lo=CONFIG.audio.kick_freq_low,
                             kick_hi=CONFIG.audio.kick_freq_high
                         ))
    preprocess_metadata = engine.preprocess()
    logger.log_params("render_preprocess_cache", preprocess_metadata, 
                     code_versions={
                         "processors": CODE_VERSIONS["processors"], 
                         "render_engine": CODE_VERSIONS["render_engine"]
                     })
    
    # Generate dial-based premaster variants (this is what we keep)
    variant_dir = os.path.join(workspace_paths.outputs, "premasters")
    render_options = RenderOptions(
        target_peak_dbfs=CONFIG.audio.render_peak_target_dbfs, 
        bit_depth=CONFIG.audio.default_bit_depth, 
        hpf_hz=None, 
        save_headroom_first=False
    )
    
    variant_metadata = engine.commit_variants(variant_dir, processing_plan, opts=render_options)
    for meta in variant_metadata:
        register_and_log_artifact(manifest, logger, meta["out_path"], 
                                 kind="premaster", params=meta, 
                                 stage=f"variant__{os.path.basename(meta['out_path'])}")
    
    print(f"✅ Created {len(variant_metadata)} premaster variants")
    print("🗑️  Skipped creating single 'premaster' folder - using variants for mastering instead")
    
except Exception as e:
    print(f"❌ Pre-mastering failed: {e}")
    raise

🔧 Pre-mastering preparation - generating ALL variants...
✅ Created 37 premaster variants
🗑️  Skipped creating single 'premaster' folder - using variants for mastering instead


In [6]:
# ============================================
# MASTERING WITH VARIANT SUBFOLDERS
# ============================================

try:
    # 6) Mastering orchestrator with variant-specific subfolders
    print("🎭 Running mastering for ALL variants...")
    
    orchestrator = MasteringOrchestrator(workspace_paths, manifest)
    
    # Get recommended mastering styles (using the first variant for analysis)
    first_variant_path = variant_metadata[0]["out_path"] if variant_metadata else None
    if not first_variant_path:
        raise ValueError("No premaster variants available for mastering")
        
    premaster_analysis = analyze_wav(first_variant_path)
    mastering_styles = [
        (style, strength) for (style, strength, reason) in 
        recommend_mastering_styles_from_metrics(premaster_analysis)
    ]
    
    # Use improved local mastering provider
    providers = [LocalMasterProvider(bit_depth=CONFIG.audio.default_bit_depth)]
    logger.log_params("mastering_styles", {"styles": mastering_styles}, 
                     code_versions={"orchestrator": CODE_VERSIONS["orchestrator"]})
    
    # Run mastering for ALL variants with subfolders
    all_master_results = orchestrator.run_all_variants(
        variant_metadata=variant_metadata,
        providers=providers,
        styles=mastering_styles,
        out_tag="masters",
        level_match_preview_lufs=LEVEL_MATCH_LUFS
    )
    
    # Flatten results for compatibility with downstream code
    master_paths = []
    for variant_name, results in all_master_results.items():
        master_paths.extend([result.out_path for result in results])
    
    total_variants = len(all_master_results)
    total_masters = len(master_paths)
    print(f"✅ Created {total_masters} mastered files across {total_variants} variants")
    print(f"📁 Each variant has its own subfolder: masters/{'{variant_name}'}/")
    print(f"📝 Each subfolder contains: {', '.join([f'{s}_{int(st*100)}.wav' for s, st in mastering_styles])}")
    print("🎯 No single 'premaster' folder needed - mastering directly from variant files!")
    
except Exception as e:
    print(f"❌ Mastering failed: {e}")
    raise

🎭 Running mastering for ALL variants...
🎭 Mastering variant: PM_Transparent
  ✅ Created 4 mastered versions for PM_Transparent
🎭 Mastering variant: PM_BalancedGentle
  ✅ Created 4 mastered versions for PM_BalancedGentle
🎭 Mastering variant: PM_Bassy
  ✅ Created 4 mastered versions for PM_Bassy
🎭 Mastering variant: PM_DeepBass
  ✅ Created 4 mastered versions for PM_DeepBass
🎭 Mastering variant: PM_ModernLow-End
  ✅ Created 4 mastered versions for PM_ModernLow-End
🎭 Mastering variant: PM_Punchy
  ✅ Created 4 mastered versions for PM_Punchy
🎭 Mastering variant: PM_TightAndPunchy
  ✅ Created 4 mastered versions for PM_TightAndPunchy
🎭 Mastering variant: PM_AggressivePunch
  ✅ Created 4 mastered versions for PM_AggressivePunch
🎭 Mastering variant: PM_Clear
  ✅ Created 4 mastered versions for PM_Clear
🎭 Mastering variant: PM_ClarityAndAir
  ❌ Error processing local/neutral for PM_ClarityAndAir: Failed to create variant level-matched preview: System error.


KeyboardInterrupt: 

In [ ]:
# ============================================
# STREAMING SIMULATION & REPORTING
# ============================================

try:
    # 7) Streaming normalization simulation
    streaming_paths = []
    streaming_dataframe = None
    
    if ENABLE_STREAMING_SIMULATION and master_paths:
        print("📱 Simulating streaming platforms...")
        stream_output_dir = os.path.join(workspace_paths.outputs, "stream_previews")
        
        # Use new configuration for streaming profiles
        streaming_profiles = CONFIG.streaming.get_platform_profiles()
        
        streaming_paths, streaming_dataframe = simulate_and_export_for_platforms(
            input_path=master_paths[0],
            out_dir=stream_output_dir,
            profiles=default_streaming_profiles(),  # TODO: Update this to use CONFIG
            bit_depth=CONFIG.audio.default_bit_depth,
            register_to_manifest=(manifest, "stream_sim")
        )
        
        logger.log_event("stream_sim_summary", {
            "platforms": len(streaming_profiles),
            "files_created": len(streaming_paths)
        })
        print(f"✅ Created {len(streaming_paths)} streaming previews")
    
    # 8) Enhanced comparison reporting
    print("📊 Generating comparison report...")
    
    # Compare original mix with some sample masters (no premaster reference)
    sample_masters = master_paths[:5]  # First 5 masters for comparison
    comparison_files = [mix_path] + sample_masters
    comparison_files = [path for path in comparison_files if os.path.exists(path)]
    
    # Use new configuration for reporting
    compare_config = CompareConfig(
        preview_seconds=CONFIG.reporting.preview_duration_seconds,
        nfft=CONFIG.analysis.default_nfft,
        reference_name=REFERENCE_NAME
    )
    
    # Create enhanced reports directory
    reports_dir = os.path.join(workspace_paths.reports, "assets")
    os.makedirs(reports_dir, exist_ok=True)
    
    # Generate comprehensive report bundle
    report_bundle = write_report_bundle(
        file_paths=comparison_files,
        reports_dir=reports_dir,
        cfg=compare_config,
        manifest=manifest,
        report_name=CONFIG.reporting.report_name,
        extra_notes="Generated by cleaned up post-mix pipeline - no premaster folder, direct variant mastering"
    )
    
    # Enhanced HTML report with configuration info
    try:
        enhanced_html = write_report_html_enhanced(
            report_bundle["summary_df"], 
            report_bundle["deltas_df"], 
            report_bundle["plots"],
            os.path.join(reports_dir, "comparison_report_enhanced.html"),
            title="Post-Mix Analysis Report (Cleaned)",
            extra_notes="Generated with improved pipeline - variant-based mastering with organized subfolders",
            code_versions=CODE_VERSIONS,
            dial_snapshot=processing_plan[1][1].__dict__ if len(processing_plan) > 1 else {}
        )
        register_and_log_artifact(manifest, logger, enhanced_html, 
                                 kind="report", params={"enhanced": True, "version": "2.0"}, 
                                 stage="enhanced_report")
    except Exception as e:
        print(f"⚠️ Enhanced report generation failed (continuing): {e}")
    
    print("✅ Reports generated successfully")
    print("📝 Note: Comparison includes original mix + sample masters (no premaster files)")
    
except Exception as e:
    print(f"❌ Reporting failed: {e}")
    raise

In [ ]:
# ============================================
# FINALIZATION AND SUMMARY
# ============================================

try:
    # 9) Create reproducibility bundle
    print("📦 Creating reproducibility bundle...")
    
    bundle_path = os.path.join(workspace_paths.reports, "bundles", f"{logger.run_id}.zip")
    
    # Only include master files and streaming previews (no premaster files)
    all_outputs = list(set(master_paths + streaming_paths))
    
    reproducibility_zip = make_repro_zip(
        bundle_path,
        workspace_root=workspace_paths.root,
        run_logger=logger,
        env_info=environment_info,
        inputs=[mix_path],
        outputs=all_outputs,
        extra_jsons={
            "code_versions": CODE_VERSIONS, 
            "recommendations": [(name, asdict(dial_state)) for (name, dial_state) in processing_plan],
            "configuration": CONFIG.to_dict(),  # Include configuration snapshot
            "pipeline_version": "2.0_cleaned_no_premaster",
            "variant_structure": {
                "total_variants": len(processing_plan),
                "total_masters": len(master_paths),
                "folder_structure": "masters/{variant_name}/{style}_{strength}.wav",
                "note": "No premaster folder - mastering done directly from premasters variants"
            }
        },
        readme_text="Bundle created by cleaned up post-mix pipeline with variant-specific master subfolders. No single premaster folder - mastering done directly from premasters variants."
    )
    
    register_and_log_artifact(manifest, logger, reproducibility_zip, 
                             kind="bundle", params={"run_id": logger.run_id, "version": "2.0"}, 
                             stage="repro_zip")
    
    # 10) Write final summary with enhanced logging
    logger.write_summary({
        "pipeline_version": "2.0_cleaned_with_variants_no_premaster",
        "project": PROJECT_NAME,
        "run_id": logger.run_id,
        "input_file": mix_path,
        "total_variants": len(processing_plan),
        "total_premaster_variants": len(variant_metadata),
        "total_master_files": len(master_paths),
        "variant_folder_structure": "masters/{variant_name}/{style}_{strength}.wav",
        "streaming_previews": streaming_paths,
        "main_report": report_bundle["html_path"],
        "configuration_snapshot": CONFIG.to_dict(),
        "environment_hash": json_sha256(environment_info),
        "note": "No premaster folder created - mastering done directly from variants"
    })
    
    write_manifest(manifest)
    
    print("\n" + "="*60)
    print("🎉 PROCESSING COMPLETED SUCCESSFULLY - ALL VARIANTS!")
    print("="*60)
    print(f"📁 Workspace: {workspace_paths.root}")
    print(f"🎵 Original: {mix_path}")
    print(f"\n📋 Generated {len(processing_plan)} processing variants:")
    for i, (variant_name, _) in enumerate(processing_plan[:5], 1):  # Show first 5
        print(f"   {i}. {variant_name}")
    if len(processing_plan) > 5:
        print(f"   ... and {len(processing_plan)-5} more variants")
    
    print(f"\n📁 Folder structure (streamlined - no premaster folder):")
    print(f"   📁 outputs/")
    print(f"   📁   ├── premasters/          ({len(variant_metadata)} variant files)")
    print(f"   📁   └── masters/             ({len(all_master_results)} variant subfolders)")
    
    print(f"\n🎭 Masters organized in subfolders ({len(all_master_results)} variant folders):")
    print(f"   📁 masters/")
    sample_variants = list(all_master_results.keys())[:3]
    for variant in sample_variants:
        print(f"   📁   ├── {variant}/")
        sample_files = [os.path.basename(r.out_path) for r in all_master_results[variant][:2]]
        for file in sample_files:
            print(f"   📄      ├── {file}")
            print(f"   📄      ├── {file.replace('.wav', '__LM-14LUFS.wav')}")  # Level matched version
        if len(all_master_results[variant]) > 2:
            print(f"   📄      └── ... and {len(all_master_results[variant])-2} more files")
    
    if len(all_master_results) > 3:
        print(f"   📁   └── ... and {len(all_master_results)-3} more variant folders")
    
    if streaming_paths:
        print(f"\n📱 Streaming previews ({len(streaming_paths)}):")
        for i, path in enumerate(streaming_paths[:3], 1):  # Show first 3
            print(f"   {i}. {os.path.basename(path)}")
        if len(streaming_paths) > 3:
            print(f"   ... and {len(streaming_paths)-3} more")
    
    print(f"\n📊 Main report: {report_bundle['html_path']}")
    print(f"📦 Reproducibility bundle: {reproducibility_zip}")
    print("\n✨ Pipeline completed with streamlined folder structure!")
    print("🗑️  No 'premaster' folder created - cleaner output organization")
    print("🎯 Each variant has its own master subfolder with all mastering styles")
    
except Exception as e:
    print(f"❌ Finalization failed: {e}")
    raise

## Summary of Improvements

This cleaned up version includes:

### ✅ **Code Quality Improvements**
- Removed duplicate mastering provider classes
- Consolidated audio conversion logic into `audio_utils.py`
- Fixed naming inconsistencies throughout codebase
- Eliminated redundant functions and utilities

### ✅ **Configuration Management**
- Centralized configuration system in `config.py`
- Environment variable support for key settings
- Consistent parameter usage across all modules
- Configuration versioning and snapshots

### ✅ **Error Handling**
- Proper exception handling with specific error types
- Input validation for all audio processing functions
- Graceful failure recovery where possible
- Detailed error messages for debugging

### ✅ **Architecture Improvements**
- Simplified over-engineered abstractions
- Cleaner separation of concerns
- More maintainable and readable code structure
- Better documentation and type hints

### 🔧 **Next Steps for Further Improvement**
- Add unit tests for all modules
- Implement async processing for better performance
- Add more streaming platform profiles
- Create CLI interface for batch processing
- Add real-time monitoring and progress bars